In [1]:
from utils import 
from path_settings import MODEL_SAVE_PATH, DATA_PATH, LOG_PATH
from torch.utils.tensorboard import SummaryWriter
import SimulateDatasets.GenTrainingData as g
from utils import create_splits, get_dataloaders, force_remove_dir
from Network_models import Trainer as t
from importlib import reload
import os
reload(t)
reload(g)
import datetime


In [2]:
cell_type = "GRU"
configs = {
    "model_specs": {
        "model_name": "CustomRNN", # To diffrentiate from the several types of RNN cells.
        "model_path": "Network_models.RNN_models",
        "model_params": {
            "input_size": 4,
            "hidden_size": 64,
            "num_layers": 1,
            "output_size": 3,
            "cell_type": cell_type,
        }
    },

    "device": "cuda",
    "optimizer_specs": {
        "optimizer_name": "Adam",
        "optimizer_params": {
            "lr": 0.001
        }
    },

    "distance_loss": "geodesic",
    "regularisation_loss": "L2",
    "distance_weight": 1,

    "save_path": MODEL_SAVE_PATH + f"\\{cell_type}_model",
    "log_path": LOG_PATH,
    "check_path": MODEL_SAVE_PATH + f"\\{cell_type}_model_checkpoints",

    "training_config": {
        "task_id": "0.1q",
        "batch_size": 128,
        "mini_batch_size": 32,
        "seq_len": 1000,
        "prep_phase": 500,
    }
}

configs["training_config"]["data_save_path"] = DATA_PATH + ("\\Data_" + configs["training_config"]["task_id"] + ".pth")
configs["save_path"] = configs["save_path"] + ("_" + configs["training_config"]["task_id"])
configs["check_path"] = configs["check_path"] + ("_" + configs["training_config"]["task_id"])
configs["log_path"] = configs["log_path"] + ("\\Log_" + configs["training_config"]["task_id"])

for path in [configs["save_path"], configs["check_path"], configs["log_path"]]:
    os.makedirs(path, exist_ok=True)
    
data1 = g.gen_training_data(configs['training_config'])
dataloaders = get_dataloaders(data1, batch_size=configs["training_config"]["mini_batch_size"], k=5)

In [3]:
rnn_trainer = t.Trainer(configs)
rnn_trainer.load_model(configs["save_path"] + "_1.pth")

In [4]:
rnn_trainer.save_model(configs["save_path"] + f"\\model_1.pth", full = 1)

In [7]:
rnn_trainer = t.Trainer(configs)
for i, (train_loader, test_loader) in enumerate(dataloaders):
    print(f"Training on split {i+1}")
    if i:
        rnn_trainer.refresh()
    new_log_path = configs['log_path'] + f"\\{cell_type}_model_{configs['training_config']['task_id']}_{i+1}"
    force_remove_dir(new_log_path)
    
    rnn_trainer.train(train_loader, 
                      test_loader, 
                      epochs=100, 
                      save_path = configs["save_path"] + f"\\model_{i+1}.pth", 
                      check_path= configs['check_path'] + f"\\model_{i+1}", 
                      log_path = new_log_path)


Training on split 1
Successfully removed directory: D:\Projects\mental-rotations\logs\Log_0.1q\GRU_model_0.1q_1
Training on split 2
Successfully removed directory: D:\Projects\mental-rotations\logs\Log_0.1q\GRU_model_0.1q_2
Training on split 3
Successfully removed directory: D:\Projects\mental-rotations\logs\Log_0.1q\GRU_model_0.1q_3
Training on split 4
Successfully removed directory: D:\Projects\mental-rotations\logs\Log_0.1q\GRU_model_0.1q_4
Training on split 5
Successfully removed directory: D:\Projects\mental-rotations\logs\Log_0.1q\GRU_model_0.1q_5
